In [57]:
%run imports.ipynb

imports loaded.


### Func def

#### pre-processing

In [23]:
print('\nPre-Processing func:')
print('********************')

Pre-Processing func:
********************


In [8]:
def get_height(item):
    
    '''return height in cm'''
    
    item = str(item)
    
    if item[0] == '-':
        return None
    else:
        return round(30.48 * float(item[0:item.find("'")]) + 2.54 * float(item[item.find("'")+1:item.find('"')]),2)

print('    get_height(item)')    

get_height(item)


In [9]:
def get_weight(item):
    
    '''return weight in kg'''
    
    item = str(item)
    
    if item[0] == '-':
        return None
    else:
        return 0.45 * float(item[0:item.find('lbs.')])
    
print('    get_weight(item)')    

get_weight(item)


In [11]:
def get_reach(item):
    
    '''return reach in cm'''
    
    item = str(item)
    
    if item[0] == '-':
        return None
    else:
        return 2.54 * float(item[0:item.find('"')])
    
print('    get_reach(item)')    

get_reach(item)


In [10]:
def get_std_reach(item, height, reach_list, height_list):
    '''return "standard reach" ( =mean(reach/height)*height )'''
    return round(height*np.mean(reach_list/height_list),2)

print('    get_std_reach(item, height, reach_list, height_list)')

get_std_reach(item, height, reach_list, height_list)


In [12]:
def get_age(item):
    
    '''return age (instead of date of birth)'''
    
    from datetime import date,timedelta
    
    item = str(item)
    current_date = date.today().year
    
    if item[0] == '-':
        return None
    else:
        return (current_date - int(item[item.find(',')+1:]))
    
print('    get_age(item)')    

get_age(item)


In [13]:
def get_std_age(item, age_list):
    
    '''return standrd age per weight class'''  
    return round(np.mean(age_list),2)

print('    get_std_age(item, age_list)')

get_std_age(item, age_list)


In [22]:
def get_win_perc(name, date, df):
    
    '''return current career winning %'''
    
    wins = len(df[(df['w_fighter'] == name) & (df.date < date)])
    losses = len(df[(df['l_fighter'] == name) & (df.date < date)])
    
    if wins == 0:
        return 0
    return wins/(wins+losses)

print('    get_win_perc(name, date, df)')

get_win_perc(name, date, df)


In [17]:
def get_lose_streak_data_frame(name, date, df):
    
    '''return losses data frame of: sequence(from last to first), result(win/lose) & opponent '''
    
    lose_streak = pd.DataFrame({
        'result': np.array('lose'),
        'date': pd.to_datetime(np.array(df.date[df.l_fighter == name])),
        'opponent': np.array(df['w_fighter'][df.l_fighter == name])
    })
    
    return lose_streak[lose_streak.date < date]

print('    get_lose_streak_data_frame(name, date, df)')

get_lose_streak_data_frame(name, date, df)


In [18]:
def get_win_streak_data_frame(name, date, df):
    
    '''return wins data frame of: sequence(from last to first), result(win/lose) & opponent '''
    
    win_streak = pd.DataFrame({
        'result': np.array('win'),
        'date': pd.to_datetime(np.array(df.date[df.w_fighter == name])),
        'opponent': np.array(df['l_fighter'][df.w_fighter == name])
    })
    
    return win_streak[win_streak.date < date]

print('    get_win_streak_data_frame(name, date, df)')

get_win_streak_data_frame(name, date, df)


In [19]:
def get_streak_data_frame(name, date, df):
    
    '''return losses & winns data frame sorted by date'''
    
    wins_df = get_win_streak_data_frame(name, date, df)
    losses_df = get_lose_streak_data_frame(name, date, df)
    
    return (pd.concat([wins_df, losses_df])).sort_values(by='date', ascending=False)

print('    get_streak_data_frame(name, date, df)')

get_streak_data_frame(name, date, df)


In [20]:
def get_win_streak(name, date, df):
    
    '''return int of fighter recent win streak'''
    
    streak = get_streak_data_frame(name, date, df)
    
    count = 0
    for result in streak['result']:
        if result == 'lose':
            return count
        count += 1
    
    return count

print('    get_win_streak(name, date, df)')

get_win_streak(name, date, df)


In [21]:
def get_lose_streak(name, date, df):
    
    '''return int of fighter recent losses streak'''
    
    streak = get_streak_data_frame(name, date, df)
    
    count = 0
    for result in streak['result']:
        if result == 'win':
            return count
        count += 1
    
    return count

print('    get_lose_streak(name, date, df)')

get_lose_streak(name, date, df)


In [23]:
def get_fighter_stats(name, data ):
    '''return fighters statistics from stat_df'''
    
    stat_list = data.columns[1:]
    row = (data[stat_list][data.name==name]).reset_index(drop=True)
    
    if len(row) < 1:
        return np.array(len(stat_list)*[None])
    else:
        return np.array(row.loc[0])
    
print('    get_fighter_stats(name, data )')    

get_fighter_stats(name, data )


In [24]:
def add_columns(data, column_list):
    
    '''add new columns to data '''
    
    for column in column_list:
        data[column] = None
    
    return data

print('    add_columns(data, column_list)')

add_columns(data, column_list)


#### feature_engineering

In [24]:
print('\nFeature Engineering func:')
print('*************************')

Feature Engineering func:
*************************


In [4]:
def get_stat(index, data, stat='height', fo=''):
    return data.loc[index, fo + stat]

print("    get_stat(index, data, stat='height', fo='')")

get_stat(index, data, stat='height', fo='')


In [6]:
def get_swaped_row(index, data):
    
    '''return row with opponent as winner and statistics swapped'''
    
    swapped_row = np.array([])
    
    for column in data.columns:
        
        if column.find('f_') > -1: # f_ will get opponent stats
            swapped_row = np.append(swapped_row, get_stat(index, data=df, stat=column[2:], fo='o_'))
        elif column.find('o_') > -1: # o_ will get fighter stats
            swapped_row = np.append(swapped_row, get_stat(index, data=df, stat=column[2:], fo='f_'))          
        elif column == 'result':
            swapped_row = np.append(swapped_row, 'lose')
        elif column == 'fighter':
            swapped_row = np.append(swapped_row, get_stat(index, data=df, stat='opponent'))
        elif column == 'opponent':
            swapped_row = np.append(swapped_row, get_stat(index, data=df, stat='fighter'))
        else:
            swapped_row = np.append(swapped_row, get_stat(index, data=df, stat=column))
    
    return swapped_row

print('    get_swaped_row(index, data)')

get_swaped_row(index, data)


In [1]:
def get_wins(combo, data):

    data = data[(data.result=='win') & (data.stance_combo==combo)]
    return len(data)

print('    get_wins(combo, data)')

get_wins(combo, data=stance_df)


In [2]:
def get_wins_ratio(combo, data):
    
    combo_df = data[data.stance_combo==combo]
    if len(combo_df) == 0:
        return 0
    else:
        return get_wins(combo, data=data)/len(combo_df)
    
print('    get_wins_ratio(combo, data)')    

get_wins_ratio(combo, data)


In [42]:
def get_layoff_time(fighter, current_date, data):
    '''return layoff time between current fight & last fight.'''
    
    data = data[(data.fighter==fighter)|(data.opponent==fighter)].sort_values(by='date',ascending=False)
    last_fight_date = np.max(data.date[data.date < current_date])
    
    if str(last_fight_date) == 'NaT':
            return 0
    else:    
        time_string = str(current_date - last_fight_date)
        time_dif = time_string[0:time_string.find(' days')]
        return int(time_dif)

print('    get_layoff_time(fighter, current_date, data)')

    get_layoff(fighter, current_date, data)


#### fight_stats_model_training

In [34]:
print('\nfight_stats_model_training func:')
print('********************************')


fight_stats_model_training func:
********************************


In [4]:
def get_sig_features(data, sig_r):
    '''return list of most significant columns from corr data frame'''
    sig_features = []
    for i in data.columns:
        for j in data.index.values:
            if data.loc[j,i] in sig_r:
                sig_features.append(j)
            
    return sig_features

print('    get_sig_features(data, sig_r)')

get_sig_features(data, sig_r)


In [3]:
def get_r_score(X, y):
    from scipy.stats import pearsonr
    r, _ = pearsonr(X, y)
    return r

print('    get_r_score(X, y)')

get_r_score(X, y)


In [35]:
def set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3):

    if 'set' not in data.columns:
        data.insert(0, 'set', None) # inserd 'set' column to position 0
    
    for i in range(len(df)):
        
        if data.loc[i,'result'] == None:
            data.loc[i,'set'] = 'new'
        
        elif random.random() < test_size:
            data.loc[i,'set'] = 'test'
        else:
            if random.random() < valid_size:
                data.loc[i,'set'] = 'validation'
            else:
                data.loc[i,'set'] = 'train' 

    X_train = data[X_columns][data.set=='train'] 
    X_test = data[X_columns][data.set=='test']
    y_train = data[y_columns][data.set=='train']
    y_test = data[y_columns][data.set=='test']
    X_val = data[X_columns][data.set=='validation']
    y_val = data[y_columns][data.set=='validation']

    print('Data split: Data=%s, train_set=%s, validation_set=%s, test_set=%s'  %(str(data.shape), str(X_train.shape), str(X_val.shape), str(X_test.shape)))
    return data

print('    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)')

    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)


In [39]:
def set_knn_reg(X_columns, y_columns, data, n=5):
    
    knn_reg = KNeighborsRegressor(n_neighbors=n)
    X_train = data[X_columns][data.set=='train']
    y_train = data[y_columns][data.set=='train']
    knn_reg.fit(X_train,y_train)
    return knn_reg
    
print('    set_knn_reg(X_columns, y_columns, data, n=5)')    

    set_knn_reg(X_columns, y_columns, data, n=5)


In [10]:
def set_predictions(model, data, X_columns, y_columns):
    
    for column in y_columns:
        data[column + '_pred'] = None
        data[column + '_pred'] = model.predict(data[X_columns])
    
    return data

print('    set_predictions(model, data, X_columns, y_columns)')

set_predictions(model, data, X_columns, y_columns)


In [40]:
def get_rmse(act, pred):
    return round(math.sqrt(mean_squared_error(act, pred))/(np.max(act) - np.min(act)),4)

print('    get_rmse(act, pred)')

    get_rmse(act, pred)


In [52]:
def get_grad_descent_step(alpha, X_columns, y_columns, data, n):
    row = {}
    row['neighbors'] = [n]
    
    knn_model = set_knn_reg(X_columns=X_columns, y_columns=y_columns, data=data, n=n)
    data = set_predictions(model=knn_model, X_columns=X_columns, y_columns=y_columns, data=data)
    
    total_rmse = 0
    for stat in ['kd','str','td','sub']:
        row[stat + '_dif rmse'] = [get_rmse(act=df[stat+'_dif'][df.set=='validation'], pred=df[stat+'_dif_pred'][df.set=='validation'])]
        total_rmse += np.mean(row[stat + '_dif rmse'])
        
    row['total_rmse'] = np.round(total_rmse, 4)
    return pd.DataFrame.from_dict(data=row)

print('    get_grad_descent_step(alpha, X_columns, y_columns, data, n)')    

    get_grad_descent_step(alpha, X_columns, y_columns, data, n)


#### fight_result_model_training

In [56]:
print('\nfight_result_model_training func:')
print('*********************************')


fight_result_model_training func:
*********************************


In [70]:
def set_svm_clf(X_columns, y_columns, data, C=1, gamma='scale', kernel='linear', degree=2):
    
    try:
        if kernel == 'poly':
            svm_clf = svm.SVC(kernel=kernel,C=C, gamma=gamma, degree=degree)
        else:
            svm_clf = svm.SVC(kernel=kernel,C=C, gamma=gamma)
    except:
        print('kernel acceptable values: linear/poly/rbf.')
        print('C acceptable values: 0.1 < C < 100.')
        print('gamma acceptable values: 0.0001 < gamma < 10.')
        return None
    
    svm_clf = svm.SVC(kernel=kernel,C=C, gamma=gamma, degree=degree)
    X_train = data[X_columns][data.set=='train']
    y_train = data[y_columns][data.set=='train']
    
    if len(y_columns) == 1:
        y_train = np.ravel(np.array(data[y_columns[0]][data.set=='train']).reshape(-1,1))
    else:    
        y_train = data[y_columns][data.set=='train']
    
    svm_clf.fit(X_train,y_train)
    return svm_clf

print("    set_svm_clf(X_columns, y_columns, data, C=1, gamma='scale', kernel='linear', degree=2)")    

    set_svm_clf(X_columns, y_columns, data, C=1, gamma='scale', kernel='linear', degree=2)


In [66]:
def get_model_evaluation(data):

    y_test_act = data['result'][data.set=='test']
    y_test_pred = data['result_pred'][data.set=='test']

    print('Accuracy = %.6f' % accuracy_score(y_test_act,y_test_pred))
    ConfusionMatrixDisplay.from_predictions(y_test_act,y_test_pred)  
    
print('    get_model_evaluation(data)')    

    get_model_evaluation(data)


In [73]:
def get_accuracy(act, pred):
    return accuracy_score(act, pred)

print('    get_accuracy(act, pred)')

    get_accuracy(act, pred)


In [74]:
def get_svm_grad_step(kernel, C, gamma, degree, X_columns, y_columns, data):
    
    row = {'kernel':[kernel], 'C':[C], 'gamma':[gamma], 'degree':[degree]}
    
    svm_clf = set_svm_clf(X_columns=X_columns, y_columns=y_columns, data=data, C=C, gamma=gamma, kernel=kernel, degree=degree)
    data = set_predictions(model=svm_clf, data=data, X_columns=X_columns, y_columns=y_columns)
    
    row['accuracy'] = [get_accuracy(act=data.result[data.set=='test'], pred=data.result_pred[data.set=='test'])]
    
    return pd.DataFrame.from_dict(data=row)

In [ ]:
>>>>>>>>>>>>

In [16]:
def set_forest_clf(X_columns, y_columns, n_estimators, tree_depth, data):
    
    
    forest_clf = RandomForestClassifier(max_depth=tree_depth, random_state=0, n_estimators=n_estimators)
    
    X_train = data[X_columns][data.set=='train']
    y_train = data[y_columns][data.set=='train']
    forest_clf.fit(X_train,y_train)
    
    return forest_clf

print('    set_forest_clf(X_columns, y_columns, n_estimators, tree_depth, data)')

set_forest_clf(X_columns, y_columns, n_estimators, tree_depth, data)


In [19]:
def set_knn_clf(X_columns, y_columns, data, n=5):
    
    knn_clf = KNeighborsClassifier(n_neighbors=n)
    X_train = data[X_columns][data.set=='train']
    
    if len(y_columns) == 1:
        y_train = np.ravel(np.array(data[y_columns[0]][data.set=='train']).reshape(-1,1))
    else:    
        y_train = data[y_columns][data.set=='train']
        
    knn_clf.fit(X_train,y_train)
    return knn_clf

print('    set_knn_clf(X_columns, y_columns, data, n=5)')

set_knn_clf(X_columns, y_columns, data, n=5)


In [14]:
def set_desicion_tree_clf(X_columns, y_columns, data, tree_depth):
    
    from sklearn.tree import DecisionTreeClassifier
    tree_clf = DecisionTreeClassifier(max_depth=tree_depth, random_state=1)
    X_train = data[X_columns][data.set=='train']
    y_train = data[y_columns][data.set=='train']
    tree_clf.fit(X_train,y_train)
    
    return tree_clf

print('    set_desicion_tree_clf(X_columns, y_columns, data, tree_depth)')

set_desicion_tree_clf(X_columns, y_columns, data, tree_depth)


#### prediction

In [27]:
print('\nPrediction func:')
print('****************')

Prediction func:
****************


In [18]:
def get_fighters_stats(fighter, opponent, data):
    import pandas as pd
    data1 = pd.DataFrame((data[data.name == fighter]).reset_index(drop=True).loc[0])[1:].rename(columns={0:fighter})
    data2 = pd.DataFrame((data[data.name == opponent]).reset_index(drop=True).loc[0])[1:].rename(columns={0:opponent})
    return pd.concat([data1,data2], axis=1)

print('    get_fighters_stats(fighter, opponent, data)')

get_fighters_stats(fighter, opponent, data)


In [28]:
>>>>

SyntaxError: invalid syntax (<ipython-input-28-d960544c3de0>, line 1)

In [28]:
'''
def get_stance_win_ratio():
    pass
'''

In [27]:
'''
def set_new_fight(fighter, opponent, data):
    row = {'set':None,
          'stance_win_ratio':None,
          'win_perc_dif':None,
          'win_strk_dif':None,
          'lose_strk_dif':None,
          'kd_dif':None,
          'str_dif':None,
          'td_dif':None,
          'sub_dif':None,
          'result':None,
          'result_pred':None,
          }
    return pd.DataFrame(row, index=[0])

print('set_new_fight(fighter, opponent, data)')
'''

set_new_fight(fighter, opponent, data)


In [22]:
'''
def set_new_fight(fighter, opponent, data):
    
    row = {'date': get_current_date(),
           'result': None,
           'fighter': fighter,
           'opponent': opponent,
           'win_perc_dif': get_win_perc(fighter=fighter, data=data) - get_win_perc(fighter=opponent, data=data),
           'win_strk_dif': get_win_streak(fighter=fighter, data=data) - get_win_streak(fighter=opponent, data=data),
           'lose_strk_dif': get_lose_streak(fighter=fighter, data=data) - get_lose_streak(fighter=opponent, data=data),
           'td avg._dif': get_td_avg(fighter=fighter, data=stats) - get_td_avg(fighter=opponent, data=stats),
           'sub. avg._dif': get_sub_avg(fighter=fighter, data=stats) - get_sub_avg(fighter=opponent, data=stats),
           'sapm_dif': get_sapm(fighter=fighter, data=stats) - get_sapm(fighter=opponent, data=stats),
           'slpm_dif': get_slpm(fighter=fighter, data=stats) - get_slpm(fighter=opponent, data=stats),
           'kd_dif':None,
           'str_dif':None,
           'td_dif':None,
           'sub_dif':None,
          }
    return pd.DataFrame(row, index=[0])

print('set_new_fight(fighter, opponent, data)')
'''

set_new_fight(fighter, opponent, data)


In [23]:
def get_current_date():
    from datetime import date
    import pandas as pd
    return pd.to_datetime(date.today())
print('get_current_date()')
'''
def get_win_perc(fighter, data):
    '''return current career winning %'''
    date = get_current_date()
    wins = len(data[(data['fighter'] == fighter)&(data.date < date) & (data['result']=='win')]) + len(data[(data['opponent'] == fighter)&(data.date < date) & (data['result']=='lose')])
    losses = len(data[(data['fighter'] == fighter)&(data.date < date) & (data['result']=='lose')]) + len(data[(data['opponent'] == fighter)&(data.date < date) & (data['result']=='win')])
    
    if wins == 0:
        return 0
    return wins/(wins+losses)
print('get_win_perc(fighter, data)')
'''
def get_lose_streak(fighter, data):
    '''return current lose streak'''
    date = get_current_date()
    data = ((data[(data.date < date)&((data.fighter == fighter)|(data.opponent == fighter))]).sort_values(by=['date'], ascending=False)).reset_index(drop=True)
    
    count = 0
    for i in range(0,len(data)):
        
        result = data.loc[i,'result']
        f = data.loc[i,'fighter']
        o = data.loc[i,'opponent']
        
        if ((result == 'lose')&(f == fighter))|((result == 'win')&(o == fighter)):
            count += 1
        else:
            return count
    return count
print('get_lose_streak(fighter, data)')

def get_win_streak(fighter, data):
    '''return current win streak'''
    date = get_current_date()
    data = ((data[(data.date < date)&((data.fighter == fighter)|(data.opponent == fighter))]).sort_values(by=['date'], ascending=False)).reset_index(drop=True)
    
    count = 0
    for i in range(0,len(data)):
        
        result = data.loc[i,'result']
        f = data.loc[i,'fighter']
        o = data.loc[i,'opponent']
        
        if ((result == 'win')&(f == fighter))|((result == 'lose')&(o == fighter)):
            count += 1
        else:
            return count
    return count
print('get_win_streak(fighter, data)')

def get_td_avg(fighter, data):
    '''return current takedown average'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'td avg.']
print('get_td_avg(fighter, data)')

def get_sub_avg(fighter, data):
    '''return current submission average'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'sub. avg.']
print('get_sub_avg(fighter, data)')

def get_sapm(fighter, data):
    '''return current strikes absorbed per minute'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'sapm']
print('get_sapm(fighter, data)')

def get_slpm(fighter, data):
    '''return current strikes landed per minutes'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'slpm']
print('get_slpm(fighter, data)')

get_current_date()
get_win_perc(fighter, data)
get_lose_streak(fighter, data)
get_win_streak(fighter, data)
get_td_avg(fighter, data)
get_sub_avg(fighter, data)
get_sapm(fighter, data)
get_slpm(fighter, data)


In [25]:
def get_winner(fighter, opponent, data):
    '''return winner of the fight'''
    winner = None
    data = (data[(data['fighter'] == fighter) & (data['opponent'] == opponent)]).reset_index(drop=True)
    
    if data.loc[0,'result_pred'] == 'win':
        winner = fighter
    else: 
        winner = opponent    
    
    return winner

print('get_winner(fighter=fighter, opponent, data)')

get_winner(fighter=fighter, opponent, data)
